In [1]:
!pip3 install geopy

In [2]:
!ls -alt
!pwd
!date

total 1452
-rw-------  1 lance lance  48657 Jul  8 20:01 nohup.out
drwxrwxr-x 10 lance lance   4096 Jul  8 19:59 .
-rw-rw-r--  1 lance lance  11483 Jul  8 19:59 LocalNomServerCreatingMayHotels.ipynb
-rw-rw-r--  1 lance lance 609474 Jul  8 19:57 Bexar_May25_receipts_scaled_map.html
-rw-rw-r--  1 lance lance   3334 Jul  8 19:56 prevBexar_May25_receipts_scaled_map.html
drwxrwxr-x  4 lance lance   4096 Jul  8 15:30 Map_Chart_Datasets
drwxrwxr-x  2 lance lance   4096 Jul  8 15:20 .ipynb_checkpoints
-rw-rw-r--  1 lance lance 609474 Jul  8 15:16 Bexar_May24_receipts_scaled_map.html
drwxrwxr-x 25 lance lance   4096 Jul  7 11:29 Learn-Python-by-Building-Data-Science-Applications
drwxrwxr-x 11 lance lance   4096 Jul  6 15:26 ..
drwxrwxr-x 10 lance lance   4096 Mar 20 22:39 esphome
drwxrwxr-x  3 lance lance   4096 Feb 13 14:37 new_esphome_work
-rw-rw-r--  1 lance lance   9785 Oct 23  2024 TexasWARNReview.ipynb
-rw-rw-r--  1 lance lance 143099 Oct 23  2024 23_Oct_24_Texas_Warn.pkl
drwxrwxr-x 20 la

In [3]:
from geopy.geocoders import Nominatim
from geopy.adapters import AioHTTPAdapter
import pandas as pd
import time

df = pd.read_csv("./Map_Chart_Datasets/filtered_BexarMay25_hotel_tax_receipts.csv")

# Point to your local server
geolocator = Nominatim(
    user_agent="tx_hotels_map",
    domain="localhost:8080",
    scheme="http"
)

def get_coordinates(row):
    full_address = f"{row['Location_Address']}, {row['Location_City']}, TX {row['Location_Zip']}"
    try:
        location = geolocator.geocode(full_address, timeout=3)
#        time.sleep(1)  # Respect rate limits even locally
        if location:
            return pd.Series([location.latitude, location.longitude])
    except Exception as e:
        print(f"Error with address '{full_address}': {e}")
        return pd.Series([None, None])

df[["Latitude", "Longitude"]] = df.apply(get_coordinates, axis=1)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Taxpayer_Number            637 non-null    int64  
 1   Taxpayer_Name              637 non-null    object 
 2   Taxpayer_Address           637 non-null    object 
 3   Taxpayer_City              637 non-null    object 
 4   Taxpayer_State             637 non-null    object 
 5   Taxpayer_Zip               637 non-null    int64  
 6   Taxpayer_County            637 non-null    int64  
 7   Taxpayer_Phone             637 non-null    object 
 8   Location_Number            637 non-null    int64  
 9   Location_Name              637 non-null    object 
 10  Location_Address           637 non-null    object 
 11  Location_City              637 non-null    object 
 12  Location_State             637 non-null    object 
 13  Location_Zip               637 non-null    int64  

In [5]:
!pip3 install folium

In [6]:
!date

Tue Jul  8 08:01:44 PM CDT 2025


In [7]:
import folium

# Create the base map centered on San Antonio
hotel_map = folium.Map(location=[29.5187, -98.6047], zoom_start=12)

# Normalize receipts to set radius scale
min_receipt = df["Total_Room_Receipts"].min()
max_receipt = df["Total_Room_Receipts"].max()
range_receipt = max_receipt - min_receipt

# Add scaled circle markers
for _, row in df.dropna(subset=["Latitude", "Longitude"]).iterrows():
    # Scale radius between 4 and 20
    scaled_radius = 4 + 16 * ((row["Total_Room_Receipts"] - min_receipt) / range_receipt)

    # Color by receipt amount (simple logic: low = light, high = dark blue)
    if row["Total_Room_Receipts"] == 0:
        color = "gray"
    elif row["Total_Room_Receipts"] < 100000:
        color = "lightblue"
    elif row["Total_Room_Receipts"] < 300000:
        color = "blue"
    else:
        color = "darkblue"

    popup_text = f"""
    <b>{row['Location_Name']}</b><br>
    Receipts: ${row['Total_Room_Receipts']:,.2f}<br>
    Units: {row['Unit_Capacity']}
    """

    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=scaled_radius,
        color=color,
        fill=True,
        fill_color=color,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(hotel_map)

# Save the map
hotel_map.save("Bexar_May25_receipts_scaled_map_2.html")


In [8]:
!date

Tue Jul  8 08:01:45 PM CDT 2025


In [9]:
!ls
!pwd

23_Oct_24_Texas_Warn.pkl
Bexar_May24_receipts_scaled_map.html
Bexar_May25_receipts_scaled_map_2.html
Bexar_May25_receipts_scaled_map.html
Bexar_Records
esphome
Learn-Python-by-Building-Data-Science-Applications
LocalNomServerCreatingMayHotels.ipynb
Map_Chart_Datasets
new_esphome_work
nohup.out
Pandas-Cookbook-Second-Edition
prevBexar_May25_receipts_scaled_map.html
Prison_Notebook
TexasWARNReview.ipynb
/var/tmp/ldh/repos
